In [ ]:
from google.colab import files
import os

print("Please upload your kaggle.json file:")
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Downloading and unzipping dataset...")
!kaggle datasets download -d grassknoted/asl-alphabet
!unzip -q asl-alphabet.zip -d asl_dataset
print("Dataset ready.")

Please upload your kaggle.json file:


Saving kaggle.json to kaggle (1).json
Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)
replace asl_dataset/asl_alphabet_test/asl_alphabet_test/A_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace asl_dataset/asl_alphabet_test/asl_alphabet_test/B_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace asl_dataset/asl_alphabet_test/asl_alphabet_test/C_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
y

Dataset ready.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
import numpy as np
import os

base_dir = 'asl_dataset/asl_alphabet_train/asl_alphabet_train'
IMG_HEIGHT = 100
IMG_WIDTH = 100
BATCH_SIZE = 128

data_augmentation = Sequential(
  [
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2),
  ],
  name="data_augmentation",
)

print("Loading training data...")
train_ds = tf.keras.utils.image_dataset_from_directory(
  base_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE
)

print("Loading validation data...")
val_ds = tf.keras.utils.image_dataset_from_directory(
  base_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Found {num_classes} classes: {class_names[:5]}...")

AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
  image = tf.cast(image, tf.float32) / 255.0
  return image, label

train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                        num_parallel_calls=AUTOTUNE)

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

print("Data pipeline (No-Cache) optimized and ready.")

Loading training data...
Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Loading validation data...
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.
Found 29 classes: ['A', 'B', 'C', 'D', 'E']...
Data pipeline (No-Cache) optimized and ready.


In [ ]:
model = Sequential([
    # First convolution block
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),

    # Second convolution block
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Third convolution block
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Flatten the results to feed into a dense layer
    Flatten(),

    # Dense (fully-connected) layer
    Dense(512, activation='relu'),
    Dropout(0.5), # dropout for regularization

    # Output layer
    Dense(num_classes, activation='softmax') # 'softmax' for multi-class classification
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     6,554,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,662,237 (25.41 MB)

 Trainable params: 6,662,237 (25.41 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 20

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

print("Training finished.")

Starting model training...
Epoch 1/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 155s 274ms/step - accuracy: 0.1371 - loss: 2.9710 - val_accuracy: 0.4750 - val_loss: 1.6599
Epoch 2/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 143s 263ms/step - accuracy: 0.4435 - loss: 1.7259 - val_accuracy: 0.6351 - val_loss: 1.0888
Epoch 3/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 143s 262ms/step - accuracy: 0.6078 - loss: 1.1693 - val_accuracy: 0.7609 - val_loss: 0.7465
Epoch 4/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 144s 264ms/step - accuracy: 0.6973 - loss: 0.8826 - val_accuracy: 0.8569 - val_loss: 0.4685
Epoch 5/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 143s 261ms/step - accuracy: 0.7504 - loss: 0.7218 - val_accuracy: 0.9117 - val_loss: 0.3002
Epoch 6/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 142s 261ms/step - accuracy: 0.7866 - loss: 0.6094 - val_accuracy: 0.9013 - val_loss: 0.3003
Epoch 7/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 142s 260ms/step - accuracy: 0.8162 - loss: 0.5238 - val_accuracy: 0.9186 - val_loss: 0.2607
Epoch 8/20
544/544 ━━━━━━━━━━━━━━━━━━━━ 142s 261m

In [ ]:
model.save('asl_model.h5')

print("Model saved as asl_model.h5. You can now download it.")

Model saved as asl_model.h5. You can now download it.
